In [ ]:
###Main page: PubmedApp

from flask import Flask, render_template, request, redirect
from bokeh.embed import components
#import PubDatePlotting as pdp
#import StateGraph as sg

# output_notebook()

app = Flask(__name__)

app.vars = {}

@app.route('/', methods=['GET', 'POST'])
def index():
    # nquestions=app_lulu.nquestions
    if request.method == 'GET':
        return render_template('pubsearch.html')
    else:
        
        del app.vars

#         app.vars['cancertype'] = request.form['cancertype']

#         p1 = pdp.pubByDate(app.vars['cancertype'], False)
#         p2 = pdp.pubByDate(app.vars['cancertype'], True)
#         p3, p4 = stateGraph(app.vars['cancertype'])
#         plots = {'p1': p1, 'p2': p2, 'p3': p3, 'p4': p4}
#         script, div = components(plots)

        return render_template('pubreturn.html', script=script, div=div, ctype=app.vars['cancertype'])



@app.route('/search', methods=['GET', 'POST'])
def searchView():
    if request.method == 'GET':
        app.vars = {}
        return render_template('pubsearch.html')
    else:
        app.vars = {}
        app.vars['searchStr'] = request.form['searchterm']
        return countsView()


        
@app.route('/counts', methods=['POST'])
def countsView():
    if 'countsData' not in app.vars:
        
        app.vars['countsData'] = True
        app.stateplot, app.stateplotnorm = stateGraph(app.vars['searchStr'])
        plots = {'stateplot': app.stateplot, 'stateplotnorm': app.stateplotnorm}
        script, div = components(plots)
        return render_template('pubview.html', script=script, div=div)
    else:
        
        plots = {'stateplot': app.stateplot, 'stateplotnorm': app.stateplotnorm}
        script, div = components(plots)
        return render_template('pubview.html', script=script, div=div)

    
    
    
    
@app.route('/geo', methods=['POST'])
def geoView():
    if 'geoData' not in app.vars:
        app.vars['geoData'] = True
        return render_template('pubview.html')
    else:
        return render_template('pubview.html')

    
    
    
@app.route('/auth_inst', methods=['POST'])
def auth_instView():
    if 'auth_instData' not in app.vars:
        app.vars['auth_instData'] = True
        return render_template('pubview.html')
    else:
        return render_template('pubview.html')

if __name__ == "__main__":
    app.run(port=33507)

 * Running on http://127.0.0.1:33507/ (Press CTRL+C to quit)
127.0.0.1 - - [17/May/2017 22:33:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2017 22:33:31] "GET /static/style.css HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2017 22:33:31] "GET /static/loading.gif HTTP/1.1" 200 -
[2017-05-17 22:33:39,379] ERROR in app: Exception on /search [POST]
Traceback (most recent call last):
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/anaconda/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_req

In [83]:
#########ASYNCIO version of StateGraph

# StateGraph -  used to pull data from pubmed through an api
from aiohttp import ClientSession
import asyncio
import urllib
from urllib.parse import quote
#import concurrent.futures
#import requests

import json
from bokeh.sampledata import us_states
from bokeh.plotting import figure, show, output_notebook
import pickle

dataDir = "./static/"
moneyFile = "FundingPerState2016.pkl"

# affiliation = AD
searchField = "[AD]"
us_states = us_states.data.copy()
del us_states["HI"]
del us_states["AK"]

state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.text()

###add states correlating with responses############
async def run(states, ss):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    async with ClientSession() as session:
        for state in states:
            tu = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term="+ss+"&mindate=2012/01/01&maxdate=2016/12/31&usehistory=y&retmode=json"
            task = asyncio.ensure_future(fetch(tu, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        
        # you now have all response bodies in this variable
        #responses can be converted to json, originally were strings
        #print(json.loads(responses[0]))
    return responses

# gets data from each state while string : ss=searchstring
def getStates(ss):
    #start threads and create queue of URLs
    loop = asyncio.get_event_loop()
    states = [us_states[state]["name"] for state in us_states]
    future = asyncio.ensure_future(run(states,ss))
    res = loop.run_until_complete(future)
    #print(res)
    for idx, state in enumerate(us_states):
        search_data = json.loads(res[idx])
        #webenv = search_data["esearchresult"]['webenv']
        total_records = int(search_data["esearchresult"]['count'])
        us_states[state]["count"] = total_records
        

def stateGraph(si):
    ss = urllib.parse.quote(si)
    p = figure(title=si + " Cancer Publications",
               toolbar_location="left", plot_width=800, plot_height=510)
    p2 = figure(title=si + " Cancer Publications (Normalized by funding)",
                toolbar_location="left", plot_width=800, plot_height=510)
    p.xaxis.visible = False
    p.xgrid.visible = False
    p.yaxis.visible = False
    p.ygrid.visible = False
    
    p2.xaxis.visible = False
    p2.xgrid.visible = False
    p2.yaxis.visible = False
    p2.ygrid.visible = False

    ##FOR SEARCHING YOU WILL NEED TO ESCAPE SPACES AND SPECIAL CHARS
    getStates(ss+"+cancer")
    for state in us_states:
        print(state)
        print(us_states[state]["count"])

    # unnormalized to money version
    state_counts = [us_states[code]["count"] for code in us_states]
    state_counts_norm = state_counts
    max_state_counts = max(state_counts)
    state_counts = [x / max_state_counts for x in state_counts]

    # normalized to money
    fbs = pickle.load(open(dataDir + moneyFile, "rb"))
    state_counts_norm = [us_states[code]["count"] / fbs[us_states[code]["name"]] for code in us_states]
    max_state_counts_norm = max(state_counts_norm)
    state_counts_norm = [x / max_state_counts_norm for x in state_counts_norm]

    p.patches(state_xs, state_ys, fill_color="#377BA8", fill_alpha=state_counts,
              line_color="#884444", line_width=1.5)

    p2.patches(state_xs, state_ys, fill_color="#377BA8", fill_alpha=state_counts_norm,
               line_color="#884444", line_width=1.5)

    #show(p)
    #show(p2)
    return p, p2

#p, p2 = stateGraph("prostate")
#show(p)
#show(p2)
# stateGraph("lung")
# stateGraph("breast")



In [69]:
show(app.stateplot)